In [1]:
import pandas as pd
import numpy as np
import itertools
import random
from sklearn.model_selection import train_test_split

## Model Training

### 1. Dataset

Matches data we can get from RIOT API. For more API detail, please access this link https://developer.riotgames.com/

Our recommendation system will suggest one champion to the next drafting team, given a valid incomplete champion composition. (Here, valid means an incomplete composition that can actually occur during the drafting phase.)
Note that in the drafting process, the two teams alternate in selecting champions following a “1-2-2-2-2-1” order, during which the champions already chosen are visible to both teams.

Example: Team 1 picks first, then Team 2 will have two picks next.

In [2]:
# Sample data after processing from RIOT API
# Definition of matches
matches = [
    (["Ahri", "Lux", "LeeSin", "Ezreal", "Thresh"], ["Zed", "Kaisa", "Nami", "Riven", "Yasuo"], True),
    (["Zed", "Yasuo", "Riven", "Nami", "Lux"], ["Ahri", "LeeSin", "Thresh", "Ezreal", "Kaisa"], False),
    (["Ahri", "Lux", "Ezreal", "LeeSin", "Thresh"], ["Zed", "Kaisa", "Nami", "Riven", "Yasuo"], True),
    (["Ahri", "Lux", "Ezreal", "LeeSin", "Thresh"], ["Zed", "Kaisa", "Nami", "Riven", "Yasuo"], True),
    (["Jinx", "Lulu", "Vi", "Garen", "Annie"], ["Ezreal", "Lux", "LeeSin", "Thresh", "Ahri"], False),
    (["Zed", "Katarina", "Yasuo", "Riven", "Nami"], ["Lux", "Ezreal", "Thresh", "Vi", "Annie"], True),
    (["Ahri", "LeeSin", "Garen", "Ezreal", "Thresh"], ["Zed", "Riven", "Kaisa", "Lulu", "Vi"], False),
    (["Lux", "Ezreal", "Thresh", "Lulu", "Jinx"], ["Zed", "Yasuo", "Nami", "Riven", "LeeSin"], True),
    (["Ahri", "Kaisa", "LeeSin", "Vi", "Thresh"], ["Lux", "Ezreal", "Jinx", "Lulu", "Nami"], True),
    (["Zed", "Yasuo", "Katarina", "Vi", "Riven"], ["Ahri", "Lux", "Ezreal", "LeeSin", "Thresh"], False),
    (["Ezreal", "Lux", "LeeSin", "Thresh", "Lulu"], ["Jinx", "Nami", "Vi", "Riven", "Yasuo"], False),
    (["Ahri", "Ezreal", "Thresh", "Lulu", "LeeSin"], ["Zed", "Yasuo", "Nami", "Riven", "Lux"], True),
    (["Jinx", "Lulu", "Vi", "Annie", "Garen"], ["Ezreal", "Thresh", "LeeSin", "Lux", "Ahri"], True),
    (["Ahri", "Lux", "Ezreal", "LeeSin", "Thresh"], ["Zed", "Yasuo", "Katarina", "Vi", "Riven"], False),
    (["Zed", "Kaisa", "Nami", "Riven", "Yasuo"], ["Ahri", "Lux", "Ezreal", "LeeSin", "Thresh"], False),
    (["Ahri", "Jinx", "Lulu", "Vi", "Thresh"], ["Zed", "Katarina", "Yasuo", "Lux", "Ezreal"], True),
    (["Ezreal", "Thresh", "Lux", "Ahri", "LeeSin"], ["Riven", "Yasuo", "Zed", "Katarina", "Vi"], True),
    (["Zed", "Riven", "Yasuo", "Kaisa", "Nami"], ["Lux", "Ahri", "Ezreal", "LeeSin", "Thresh"], False),
    (["Ahri", "Lulu", "Jinx", "Vi", "Garen"], ["Zed", "Riven", "Yasuo", "Katarina", "Nami"], True),
    (["Katarina", "Zed", "Riven", "Yasuo", "Nami"], ["Lux", "Ahri", "Ezreal", "LeeSin", "Thresh"], False),
]

print(f"Total matches: {len(matches)}")
print(matches)

Total matches: 20
[(['Ahri', 'Lux', 'LeeSin', 'Ezreal', 'Thresh'], ['Zed', 'Kaisa', 'Nami', 'Riven', 'Yasuo'], True), (['Zed', 'Yasuo', 'Riven', 'Nami', 'Lux'], ['Ahri', 'LeeSin', 'Thresh', 'Ezreal', 'Kaisa'], False), (['Ahri', 'Lux', 'Ezreal', 'LeeSin', 'Thresh'], ['Zed', 'Kaisa', 'Nami', 'Riven', 'Yasuo'], True), (['Ahri', 'Lux', 'Ezreal', 'LeeSin', 'Thresh'], ['Zed', 'Kaisa', 'Nami', 'Riven', 'Yasuo'], True), (['Jinx', 'Lulu', 'Vi', 'Garen', 'Annie'], ['Ezreal', 'Lux', 'LeeSin', 'Thresh', 'Ahri'], False), (['Zed', 'Katarina', 'Yasuo', 'Riven', 'Nami'], ['Lux', 'Ezreal', 'Thresh', 'Vi', 'Annie'], True), (['Ahri', 'LeeSin', 'Garen', 'Ezreal', 'Thresh'], ['Zed', 'Riven', 'Kaisa', 'Lulu', 'Vi'], False), (['Lux', 'Ezreal', 'Thresh', 'Lulu', 'Jinx'], ['Zed', 'Yasuo', 'Nami', 'Riven', 'LeeSin'], True), (['Ahri', 'Kaisa', 'LeeSin', 'Vi', 'Thresh'], ['Lux', 'Ezreal', 'Jinx', 'Lulu', 'Nami'], True), (['Zed', 'Yasuo', 'Katarina', 'Vi', 'Riven'], ['Ahri', 'Lux', 'Ezreal', 'LeeSin', 'Thresh'], F

### 2. Approach Methodology

#### 2.1 Naive association rule method using FP-Growth

*(This method serves as a preliminary preference metric, please refer to the paper for more detailed explanations and formulations.)*

#### 2.2  Synergy and counter relationship

**Popularity**

The popularity of a champion \( c_k \) is defined as:

$$
\text{popularity}(c_k) = P(c_k \in m[0] \lor c_k \in m[1])
$$

This represents how frequently a champion appears in any match, regardless of team.


**Synergy Score**

The synergy score between two champions \( c_i \) and \( c_j \) measures how often they win together when they are on the same team.

$$
\text{synergy}(c_i, c_j) = 
\frac{|\{ m \mid (c_i, c_j \in m[0] \land m[2] = \text{True}) \lor (c_i, c_j \in m[1] \land m[2] = \text{False}) \}|}
{|\{ m \mid (c_i, c_j \in m[0]) \lor (c_i, c_j \in m[1]) \}|}
$$

Probability of winning given that both champions are on the same team.


**Counter Score**

The counter score measures how effective one champion \( c_i \) is against another \( c_j \).

$$
\text{counter}(c_i \to c_j) = 
\frac{|\{ m \mid (c_i \in m[0] \land c_j \in m[1] \land m[2] = \text{True}) \lor (c_i \in m[1] \land c_j \in m[0] \land m[2] = \text{False}) \}|}
{|\{ m \mid (c_i \in m[0] \land c_j \in m[1]) \lor (c_i \in m[1] \land c_j \in m[0]) \}|}
$$

Probability of winning given that \( c_i \) and \( c_j \) are on opposite teams.


**Why Use These Metrics**

Both synergy and counter are:

- Normalized by popularity (avoids bias toward common champions)  
- Interpretable as win probabilities  
- More meaningful for champion recommendation than raw co-occurrence counts


In [3]:
class ChampionRelations:
    def __init__(self, matches):
        """
        Initialize the ChampionRelations object.
        
        Args:
            matches (list): A list of match tuples in the form (team1, team2, team1_win)
                            where:
                              - team1, team2: lists or sets of champions
                              - team1_win: boolean indicating if team1 won
        """
        self.matches = matches

    def synergy_score(self, ci, cj):
        """
        Calculate the synergy score between two champions.
        
        Args:
            ci (str): Champion i
            cj (str): Champion j
        
        Returns:
            float or None: Win rate when both champions are on the same team,
                           or None if they never played together.
        """
        total_same_team = 0
        same_team_wins = 0

        for team1, team2, team1_win in self.matches:
            # Check if both are on the same team
            if ci in team1 and cj in team1:
                total_same_team += 1
                if team1_win:
                    same_team_wins += 1
            elif ci in team2 and cj in team2:
                total_same_team += 1
                if not team1_win:
                    same_team_wins += 1

        if total_same_team == 0:
            return None

        return same_team_wins / total_same_team

    def counter_score(self, ci, cj):
        """
        Calculate the counter score between two champions.
        
        Args:
            ci (str): Champion i
            cj (str): Champion j
        
        Returns:
            float or None: Win rate of ci against cj,
                           or None if they never faced each other.
        """
        total_opposed = 0
        ci_wins_against_cj = 0

        for team1, team2, team1_win in self.matches:
            # ci in team1, cj in team2
            if ci in team1 and cj in team2:
                total_opposed += 1
                if team1_win:
                    ci_wins_against_cj += 1
            # ci in team2, cj in team1
            elif ci in team2 and cj in team1:
                total_opposed += 1
                if not team1_win:
                    ci_wins_against_cj += 1

        if total_opposed == 0:
            return None

        return ci_wins_against_cj / total_opposed

In [4]:
# Apply on demo dataset
relations = ChampionRelations(matches)

print("Synergy between Ahri and Lux:", relations.synergy_score("Ahri", "Lux"))
print("Synergy between Zed and Yasuo:", relations.synergy_score("Zed", "Yasuo"))
print("Counter score (Ahri vs Zed):", relations.counter_score("Ahri", "Zed"))
print("Counter score (Zed vs Ahri):", relations.counter_score("Zed", "Ahri"))

Synergy between Ahri and Lux: 0.8181818181818182
Synergy between Zed and Yasuo: 0.13333333333333333
Counter score (Ahri vs Zed): 0.8571428571428571
Counter score (Zed vs Ahri): 0.14285714285714285


=> However, this method is not optimized for large datasets containing a large number of champions.

**Optimize this synergy and counter calculation**

The algorithm is proposed in paper

<p align="center">
  <img src="../images/senergy_counter_algo.png" width="600">
</p>


In [5]:
class ChampionRelationsEfficient:
    def __init__(self, matches):
        """
        Initialize the ChampionRelationsEfficient object.
        
        Args:
            matches (list): A list of tuples (team1, team2, team1_win)
        """
        self.matches = matches
        # Build the full unique champion list
        champions = sorted({champ for t1, t2, _ in matches for champ in t1 + t2})
        self.champ_index = {c: i for i, c in enumerate(champions)}
        self.champions = champions
        
        # Initialize matrices
        size = len(champions)
        self.S = np.zeros((size, size), dtype=int)   # synergy wins
        self.Ts = np.zeros((size, size), dtype=int)  # synergy total
        self.C = np.zeros((size, size), dtype=int)   # counter wins
        self.Tc = np.zeros((size, size), dtype=int)  # counter total

    def calculate(self):
        """Compute synergy and counter for all champion pairs."""
        for team1, team2, team1_win in self.matches:
            # Determine winner/loser teams
            cwinner, closer = (team1, team2) if team1_win else (team2, team1)

            # --- Synergy (same team) ---
            for team in [cwinner, closer]:
                for ca, cb in itertools.permutations(team, 2):
                    a, b = self.champ_index[ca], self.champ_index[cb]
                    self.Ts[a][b] += 1
                    if team == cwinner:
                        self.S[a][b] += 1

            # --- Counter (winner vs loser) ---
            for ca in cwinner:
                for cb in closer:
                    a, b = self.champ_index[ca], self.champ_index[cb]
                    self.C[a][b] += 1  # winner beats loser
                    self.Tc[a][b] += 1
            for ca in closer:
                for cb in cwinner:
                    a, b = self.champ_index[ca], self.champ_index[cb]
                    self.Tc[a][b] += 1  # total matchups

        # Compute normalized matrices
        synergy = np.divide(self.S, self.Ts, out=np.zeros_like(self.S, dtype=float), where=self.Ts != 0)
        counter = np.divide(self.C, self.Tc, out=np.zeros_like(self.C, dtype=float), where=self.Tc != 0)

        return synergy, counter

    def get_synergy(self, ci, cj, synergy_matrix):
        return synergy_matrix[self.champ_index[ci], self.champ_index[cj]]

    def get_counter(self, ci, cj, counter_matrix):
        return counter_matrix[self.champ_index[ci], self.champ_index[cj]]

In [6]:
# Apply on demo dataset
relations = ChampionRelationsEfficient(matches)
synergy_matrix, counter_matrix = relations.calculate()

print("Synergy(Ahri, Lux):", relations.get_synergy("Ahri", "Lux", synergy_matrix))
print("Counter(Ahri → Zed):", relations.get_counter("Ahri", "Zed", counter_matrix))
print("Counter(Zed → Ahri):", relations.get_counter("Zed", "Ahri", counter_matrix))

Synergy(Ahri, Lux): 0.8181818181818182
Counter(Ahri → Zed): 0.8571428571428571
Counter(Zed → Ahri): 0.14285714285714285


## 3 Recommendation with Synergy and counter relationship approach

### 3.1 Equally weighted sum

*(This method serves as a preliminary preference metric, please refer to the paper for more detailed explanations and formulations.)*

### 3.2 Weighted average by sample size

In this approach, we improve upon the simple average method (Approach 1) by taking into account 
the reliability of each synergy and counter score. Some pairs of champions may have very few matches 
together (small sample size), which can lead to extreme or misleading win rates. To correct this, 
we weight each score by how many times that pairing actually occurred.

The goal is to recommend a champion \( c \) that maximizes the following weighted average:

$$
\underset{c \in C, \, c \notin Allies, \, c \notin Opponents, \, c \notin Bans}{\arg\max}
\left(
\frac{
\sum_{a \in Allies} \text{synergy}(c,a) \times T_S(c,a)
+ 
\sum_{o \in Opponents} \text{counter}(c,o) \times T_C(c,o)
}{
\sum_{a \in Allies} T_S(c,a)
+
\sum_{o \in Opponents} T_C(c,o)
}
\right)
$$

Where:

| Symbol | Meaning |
|:-------|:--------|
| $ \text{synergy}(c,a) $ | Win probability when champion $c$ and ally $a$ are on the same team |
| $ \text{counter}(c,o) $ | Win probability when champion $c$ faces opponent $o$ |
| $ T_S(c,a) $ | Number of matches where $c$ and $a$ were on the same team |
| $ T_C(c,o) $ | Number of matches where $c$ faced $o$ |
| $ Allies, \; Opponents, \; Bans $ | Sets of currently selected or banned champions |

This approach rewards champion pairs that have **more reliable historical data**.  
For example, if a pair $(Ahri, Lux)$ has appeared together in 500 matches,  
its synergy score contributes more than a rare pair $(Ahri, Rell)$ that only appeared 3 times.

By weighting synergy and counter scores using $T_S$ and $T_C$,  
the system avoids bias toward small-sample outliers and provides  
a more stable and data-driven recommendation.

-   Champions with consistent and strong historical synergy will rank higher.  
-   Champions with proven advantage (high counter value) against the enemy team will be favored.  
-   This approach balances both win probability **and** data reliability,  
    giving a more robust recommendation result.

In [7]:
class ChampionRecommender:
    def __init__(self, relations):
        """
        Args:
            relations (ChampionRelationsEfficient): precomputed synergy and counter matrices.
        """
        self.relations = relations

    def recommend_weighted(self, allies, opponents, bans=None):
        """
        Implement Approach 2: Weighted average by sample size.
        
        Args:
            allies (list[str]): Current allied champions
            opponents (list[str]): Current enemy champions
            bans (list[str]): Champions that cannot be picked
            
        Returns:
            (str, float): Recommended champion and its score
        """
        if bans is None:
            bans = []
        
        synergy = self.relations.S
        counter = self.relations.C
        Ts = self.relations.Ts
        Tc = self.relations.Tc
        champions = self.relations.champions
        idx = self.relations.champ_index

        best_champ = None
        best_score = -np.inf

        for c in champions:
            if c in allies or c in opponents or c in bans:
                continue

            i = idx[c]

            # Weighted sum
            synergy_weighted = sum(
                synergy[i][idx[a]] * Ts[i][idx[a]] for a in allies if Ts[i][idx[a]] > 0
            )
            counter_weighted = sum(
                counter[i][idx[o]] * Tc[i][idx[o]] for o in opponents if Tc[i][idx[o]] > 0
            )

            # Denominator: total samples used
            total_weight = sum(Ts[i][idx[a]] for a in allies) + sum(Tc[i][idx[o]] for o in opponents)

            if total_weight == 0:
                continue

            score = (synergy_weighted + counter_weighted) / total_weight

            if score > best_score:
                best_score = score
                best_champ = c

        return best_champ, best_score

In [8]:
relations = ChampionRelationsEfficient(matches)
synergy_matrix, counter_matrix = relations.calculate()

recommender = ChampionRecommender(relations)

allies = ["Ahri", "Lux", "Ezreal"]
opponents = ["Zed", "Yasuo", "Nami"]
bans = ["LeeSin"]

best, score = recommender.recommend_weighted(allies, opponents, bans)
print(f"Recommended champion: {best} (score={score:.4f})")

Recommended champion: Thresh (score=11.5955)


## Evaluation

### 1. Supervised learning for match result prediction

*(This method serves as a preliminary preference metric, please refer to the paper for more detailed explanations and formulations.)*

### 2. Measure of upper-hand based on synergies and counters - composite win rate (CWR)

**Model Evaluation using Composite Win Rate (CWR) and Upper-Hand**

The model is evaluated based on how often its recommended champion leads to a lineup with a higher *composite win rate (CWR)* than the opponent team.

**Composite Win Rate (CWR):**

$$
CWR(A, O) =
\frac{
\sum_{a_1 \in A} \sum_{a_2 \in A, a_1 \ne a_2} synergy(a_1, a_2)
+ 
\sum_{a \in A} \sum_{o \in O} counter(o, a)
}{
\binom{|A|}{2} + |A| \times |O|
}
$$

Where:
- \( A \): allies
- \( O \): opponents
- \( synergy(a_1, a_2) \): win probability when both are on the same team  
- \( counter(o, a) \): win probability when \( a \) faces \( o \)

**Upper-Hand Metric:**

$$
upper\text{-}hand(A, O) =
\begin{cases}
1, & \text{if } CWR(A, O) > CWR(O, A) \\
0, & \text{otherwise}
\end{cases}
$$

**Evaluation Steps:**
1. Split the match data into **train** and **test** sets.  
2. Compute **synergy** and **counter** matrices from both sets.  
3. Simulate the drafting process:
   - Randomly pick a champion for the opponent team.  
   - Recommend an ally champion using the model.  
4. Repeat until both lineups are full.  
5. Compute \( CWR(A, O) \) and \( CWR(O, A) \) using test data.  
6. Assign `upper-hand = 1` if allies’ CWR is higher.  
7. Repeat for many simulations to compute the **average upper-hand**.

**Interpretation:**
- Average upper-hand > 0.5 → the model consistently builds stronger lineups.  
- The higher the value, the more effective the recommendation system.


In [9]:
import numpy as np

class Evaluation:
    def __init__(self, synergy_train, counter_train, champion_to_idx):
        # Only use matrices learned from training data
        self.synergy_train = synergy_train
        self.counter_train = counter_train
        self.champion_to_idx = champion_to_idx

    def compute_cwr(self, team_A, team_O):
        """Compute Composite Win Rate (CWR) for team_A against team_O using train matrices."""
        synergy_matrix = self.synergy_train
        counter_matrix = self.counter_train

        idx = self.champion_to_idx
        n_A = len(team_A)
        n_O = len(team_O)

        # Synergy term: how well allies work together
        synergy_sum = 0
        for i in range(n_A):
            for j in range(i + 1, n_A):
                a1, a2 = team_A[i], team_A[j]
                synergy_sum += synergy_matrix[idx[a1], idx[a2]]

        # Counter term: how well allies counter opponents
        counter_sum = 0
        for a in team_A:
            for o in team_O:
                counter_sum += counter_matrix[idx[a], idx[o]]

        denominator = (n_A * (n_A - 1) / 2) + (n_A * n_O)
        return (synergy_sum + counter_sum) / denominator if denominator > 0 else 0.0

    def compute_upper_hand(self, team_A, team_O):
        """Return 1 if team_A predicted to win (CWR higher using train data)."""
        cwr_A = self.compute_cwr(team_A, team_O)
        cwr_O = self.compute_cwr(team_O, team_A)
        return 1 if cwr_A > cwr_O else 0

In [10]:
class EvaluatorPipeline:
    def __init__(self, matches, ChampionRelationsEfficient, ChampionRecommender, Evaluation, test_size=0.2, random_state=42):
        """
        Full evaluation pipeline for synergy/counter-based champion recommendation.

        Args:
            matches: list of (team1, team2, team1_win) tuples
            ChampionRelationsEfficient: class used to build synergy/counter matrices
            ChampionRecommender: recommender class (trained from relations)
            Evaluation: class to compute CWR and upper-hand metrics
            test_size: fraction of matches for testing
            random_state: reproducibility seed
        """
        self.matches = matches
        self.ChampionRelationsEfficient = ChampionRelationsEfficient
        self.ChampionRecommender = ChampionRecommender
        self.Evaluation = Evaluation
        self.test_size = test_size
        self.random_state = random_state

    def run(self):
        """Run full pipeline: split, train, recommend, and evaluate."""
        # 1️ Split train/test
        train_matches, test_matches = train_test_split(
            self.matches, test_size=self.test_size, random_state=self.random_state
        )

        # 2️ Build relations on training data
        relations_train = self.ChampionRelationsEfficient(train_matches)
        synergy_train, counter_train = relations_train.calculate()

        # 3️ Create recommender trained on train data
        recommender = self.ChampionRecommender(relations_train)

        # 4️ Create evaluator (only train matrices are needed)
        evaluator = self.Evaluation(
            synergy_train, counter_train, relations_train.champ_index
        )

        # 5️Evaluate on test matches
        results_upper = []
        results_match_pick_matches_actual = []
        results_cwr_gain = []

        all_champs = list(relations_train.champ_index.keys())

        for m in test_matches:
            team1, team2, team1_win = m

            # Evaluate both perspectives (team1 vs team2, and vice versa)
            for allies_full, opponents in [(team1, team2), (team2, team1)]:
                if len(allies_full) < 5 or len(opponents) < 5:
                    continue

                removed_idx = random.randrange(len(allies_full))
                actual_missing = allies_full[removed_idx]
                allies_partial = [c for i, c in enumerate(allies_full) if i != removed_idx]
                bans = []

                # Try recommending
                try:
                    recommended, score = recommender.recommend_weighted(allies_partial, opponents, bans)
                except Exception:
                    # fallback random pick if recommender fails
                    available = [c for c in all_champs if c not in allies_partial + opponents + bans]
                    recommended = random.choice(available) if available else None
                    score = None

                if recommended is None:
                    continue

                allies_with_reco = allies_partial + [recommended]

                # Evaluate predicted advantage
                upper = evaluator.compute_upper_hand(allies_with_reco, opponents)
                results_upper.append(upper)

                # Check if prediction matches actual missing champ
                results_match_pick_matches_actual.append(1 if recommended == actual_missing else 0)

                # Compute improvement in team CWR
                cwr_before = evaluator.compute_cwr(allies_partial, opponents)
                cwr_after = evaluator.compute_cwr(allies_with_reco, opponents)
                results_cwr_gain.append(cwr_after - cwr_before)

        # 6Aggregate results
        avg_upper_hand = np.mean(results_upper) if results_upper else float("nan")
        match_pick_accuracy = np.mean(results_match_pick_matches_actual) if results_match_pick_matches_actual else float("nan")
        avg_cwr_gain = np.mean(results_cwr_gain) if results_cwr_gain else float("nan")

        print("\nEvaluation Summary")
        print("----------------------------")
        print(f"Evaluated picks: {len(results_upper)}")
        print(f"Average Upper-Hand: {avg_upper_hand:.4f}")
        print(f"Match Pick Accuracy: {match_pick_accuracy:.4f}")
        print(f"Average CWR Gain: {avg_cwr_gain:.4f}")

        return {
            "evaluated_picks": len(results_upper),
            "avg_upper_hand": avg_upper_hand,
            "match_pick_accuracy": match_pick_accuracy,
            "avg_cwr_gain": avg_cwr_gain,
        }

In [16]:
evaluator = EvaluatorPipeline(
    matches,
    ChampionRelationsEfficient,
    ChampionRecommender,
    Evaluation,
    test_size=0.1
)
evaluation = evaluator.run()


Evaluation Summary
----------------------------
Evaluated picks: 4
Average Upper-Hand: 0.5000
Match Pick Accuracy: 0.5000
Average CWR Gain: 0.0491
